In [14]:
import cv2     # for capturing videos
import math   # for mathematical operations
import matplotlib.pyplot as plt    # for plotting the images
%matplotlib inline
import pandas as pd
from keras.preprocessing import image   # for preprocessing the images
import numpy as np    # for mathematical operations
from keras.utils import np_utils
from skimage.transform import resize   # for resizing images
from sklearn.model_selection import train_test_split
from glob import glob
from tqdm import tqdm


In [2]:
df=pd.read_csv('datanew.csv',index_col ="gloss_split")
df.head()

,Unnamed: 0,gloss_bbox,gloss_fps,gloss_frame_end,gloss_frame_start,gloss_instance_id,gloss_signer_id,gloss_source,gloss_url,gloss_variation_id,gloss_video_id,gloss
gloss_split,,,,,,,,,,,,
train,0,"[385, 37, 885, 720]",25,-1,1,0,118,aslbrick,http://aslbricks.org/New/ASL-Videos/book.mp4,0,69241,book
train,1,"[190, 25, 489, 370]",25,-1,1,1,90,aslsignbank,https://aslsignbank.haskins.yale.edu/dictionar...,0,65225,book
train,10,"[462, 44, 949, 720]",25,-1,1,10,31,signschool,https://signstock.blob.core.windows.net/signsc...,0,7069,book
train,17,"[234, 17, 524, 414]",25,-1,1,17,36,startasl,https://s3-us-west-1.amazonaws.com/files.start...,0,7068,book
train,22,"[131, 26, 526, 480]",25,-1,1,22,59,asldeafined,https://media.asldeafined.com/vocabulary/14666...,0,7070,book


In [3]:
test = pd.DataFrame(df.loc[["test", "val"]]) 
train = pd.DataFrame(df.loc[['train']])

In [4]:
test=test.drop(test.columns[[0]],axis = 1) 
train=train.drop(train.columns[[0]],axis = 1) 

In [6]:
train=train.drop(['gloss_bbox','gloss_fps','gloss_frame_end','gloss_frame_start','gloss_instance_id','gloss_signer_id','gloss_source','gloss_url','gloss_variation_id'],axis = 1) 

In [7]:
train

,gloss_video_id,gloss
gloss_split,,
train,69241,book
train,65225,book
train,7069,book
train,7068,book
train,7070,book
...,...,...
train,16806,dissolve
train,16808,dissolve
train,16809,dissolve


In [8]:
# creating an empty list
train_image = []

# for loop to read and store frames
for i in tqdm(range(train.shape[0])):
    # loading the image and keeping the target size as (224,224,3)
    img = image.load_img('train_1/'+train['image'][i], target_size=(224,224,3))
    # converting it to array
    img = image.img_to_array(img)
    # normalizing the pixel value
    img = img/255
    # appending the image to the train_image list
    train_image.append(img)
    
# converting the list to numpy array
X = np.array(train_image)

# shape of the array
X.shapetest=test.drop(['gloss_bbox','gloss_fps','gloss_frame_end','gloss_frame_start','gloss_instance_id','gloss_signer_id','gloss_source','gloss_url','gloss_variation_id'],axis = 1) 

In [9]:
test

,gloss_video_id,gloss
gloss_split,,
test,7072,book
test,17713,drink
test,17716,drink
test,17718,drink
test,12320,computer
...,...,...
val,16429,disagree
val,16431,disagree
val,16805,dissolve


In [23]:
vidid = train['gloss_video_id'].tolist()
gls=train['gloss'].tolist()

In [24]:
vid=[]
for i in vidid:
    if(len(str(i))==5):
        vid.append(str(i))
    elif(len(str(i))==4):
        vid.append('0'+str(i))
    elif(len(str(i))==3):
        vid.append('00'+str(i))

In [ ]:
from PIL import Image as im
for i in range(0,len(vid)):
    path="/home/sharanya/Desktop/cadmus/vids/"+str(gls[i])+'/'+str(vid[i])
    os.mkdir(path);
    vidcap = cv2.VideoCapture('videos/'+str(vid[i])+'.mp4')   # capturing the video from the given path
    success,image = vidcap.read()
    img = np.array(image)
    #img = np.reshape(img, (1024, 720))
    img = im.fromarray(img)
    count = 0
    while success:
      filename = str(vid[i])+"_"+str(count)+'.jpg'
      cv2.imwrite(os.path.join(path , filename), image)     # save frame as JPEG file      
      success,image = vidcap.read()
      print('Read a new frame: ', success)
      count += 1

In [ ]:
    
# storing the images and their class in a dataframe
train_data = pd.DataFrame()
train_data['image'] = train_image
train_data['class'] = train_class

# converting the dataframe into csv file 
train_data.to_csv('UCF/train_new.csv',header=True, index=False)

In [30]:
from PIL import Image as im
    
# storing the images and their class in a dataframe
train_data = pd.DataFrame()

# storing the frames from training videos
for i in tqdm(range(train.shape[0])):
    count = 0
    videoFile = str(vid[i])
    cap = cv2.VideoCapture('/home/sharanya/Desktop/cadmus/videos/'+videoFile+'.mp4')   # capturing the video from the given path
    frameRate = 10
    x=1
    while(cap.isOpened()):
        frameId = cap.get(1) #current frame number
        ret, frame = cap.read()
        if (ret != True):
            break
        if (frameId % math.floor(frameRate) == 0):
            # storing the frames in a new folder named train_1
            filename ='train_1/' + videoFile+"_frame%d.jpg" % count;
            train_data['image'] = videoFile+"_frame%d.jpg" % count;
            train_data['class'] = train['gloss'][i]
            count+=1
            cv2.imwrite(filename, frame)
    cap.release()

100%|██████████| 5197/5197 [13:01<00:00,  6.65it/s]


FileNotFoundError: [Errno 2] No such file or directory: 'UCF/train_new.csv'

In [43]:
# getting the names of all the images
images = glob("train_1/*.jpg")
train_image = []
train_class = []
for i in tqdm(range(len(images))):
    # creating the image name
    train_image.append(images[i].split('/')[1])
    # creating the class of image
    train_class.append(gls[vid.index(images[i].split('/')[1].split('_')[0])])
    
# storing the images and their class in a dataframe
train_data = pd.DataFrame()
train_data['image'] = train_image
train_data['class'] = train_class

# converting the dataframe into csv file 
train_data.to_csv('train_new.csv',header=True, index=False)

100%|██████████| 36481/36481 [00:01<00:00, 27152.46it/s]


In [44]:
train_data.head()

,image,class
0,11037_frame2.jpg,clean
1,55854_frame0.jpg,sunset
2,03056_frame11.jpg,appointment
3,34733_frame2.jpg,man
4,65117_frame3.jpg,away


In [42]:
images[i].split('/')[1]

'13313_frame5.jpg'